# Implementation of Demand Side Unit in ASSUME Framework

This example serves as a comprehensive guide for users interested in understanding the intricacies of implementing and integrating a Demand Side Agent in the ASSUME framework. The primary aim of this session is to demystify the complex procedures involved in agent creation and market configuration, and simulation setup within the ASSUME environment. The tutorial aims to equip pyou with the skillset required to independently develop and deploy a Demand Side Agent furnished with a Rule-Based Bidding Strategy in the ASSUME framework.

**The example will delve into the following key areas:**

1. Introduction to Unit Agents and Bidding Strategy: Establishing the fundamental principles and nomenclature, for coding exercises.
2. Defining essential functions and attributes
3. Developing a Rule-Based Bidding Strategy
4. Integrating the new unit and bidding strategies in the model

# 1. Essential Concepts and Terminology in Electricity Market Modeling

ASSUME is a tool designed for researchers, planners, utilities, and anyone interested in understanding how energy markets work. It's like a user-friendly toolbox, available for free, that you can customize to fit your specific needs.

In the following figure the architecture of the framework is depicted. It can be roughly devided into two parts. On the left side, we have the market elements, and on the right side, we have the market participants, which we'll call 'units' here. These two parts are connected because the market participants make orders in the markets. Now, there's something special on the right side, shown in yellow. That's where we talk about the framework's ability to learn and adapt. But before we dive into all the technical details of ASSUME, let's first get familiar with some important terms and concepts that are the building blocks of electricity market modeling.

![Model Architecture](assume\docs\source\img\architecture.svg)

## 1.1 Unit- The Market Actor

In the ASSUME framework, a "Unit" represents an individual market actor responsible for buying or selling electricity. This actor operates according to the rules defined in its bidding strategy, effectively interacting with the market to meet specific objectives, such as demand-side management or optimal energy procurement.

## 1.2 Attributes of a Unit in ASSUME

Units within the ASSUME framework come with a set of attributes that define their operational behavior and interaction mechanisms within the market. Notably:

**Unit Operator**: This component acts as the operational hub for the Unit, responsible for ensuring that the bidding strategy is executed effectively in real-time market conditions.

This tutorial will particularly focus on configuring Demand Side Management (DSM) Units. DSM Units have the ability to dynamically adjust their electricity demand in response to market signals, adding a level of flexibility and strategic depth to their market participation.

# 2 Getting ASSUME Up and Running

Before diving into agent implementation and market modeling, it's essential to set up the ASSUME framework on your computational environment. In this tutorial, we will guide you through the installation of the ASSUME core package using Python's package installer, pip.

For comprehensive installation instructions, you can consult the official [ASSUME documentation](https://assume.readthedocs.io/en/latest/installation.html). However, this tutorial aims to cover all the necessary steps for a seamless setup, particularly tailored for a Google Colab environment.

**Installing ASSUME Core Package**

Given that Google Colab already provides an isolated environment, there's no need to create a virtual environment (`venv`). Therefore, the installation simplifies to running the following pip command:

In [1]:
pip install assume-framework

Note: you may need to restart the kernel to use updated packages.


And easy like this we have ASSUME installed. Now we can let it run.

Further we would like to access the predefined scenarios in ASSUME which are stored on the git repository. Hence, we clone the repository.

**Post-Installation Steps and Limitations**

Congratulations! You've successfully installed the ASSUME core package. With this, you are now ready to explore the functionalities that ASSUME offers for electricity market modeling. However, there are a couple of important points to note:

**Docker Functionalities and Colab Limitations**

While the core functionalities of ASSUME are now at your disposal, please be aware that Google Colab does not support functionalities that are dependent on Docker. This means that you won't be able to access predefined dashboards in this environment. To utilize these features, it's recommended to install Docker alongside ASSUME on your local machine.

**Accessing Predefined Scenarios from Git Repository**

ASSUME comes with a variety of predefined scenarios for electricity market modeling, and these are hosted on its Git repository. To access these, you can clone the repository by executing the following command:

In [2]:
!git clone https://github.com/assume-framework/assume.git

fatal: destination path 'assume' already exists and is not an empty directory.


Let the magic happen. Now you can run your first ever simulation in ASSUME. The following code naviagtes to the respective assume folder and starts the simulation example example_01b using the local database here in colab.

# 3. Develop a new Demand Side Unit

Now that you have ASSUME up and running, let's dive into the implementation of a new Unit Agent. Here, we'll be focusing on an Electrolyser unit, which has a potential of Demand Side Management (DSM) applications.

**Understanding Demand Side Management (DSM)**

Before we start coding, it's essential to understand what DSM is and why it's important in electricity market modeling. DSM allows for the dynamic adjustment of electricity demand, contributing to balanced grid operations.

**Overview of this Coding Session**

Here's a quick rundown of what we'll be covering in today's coding session:

1. **Initializing Core Attributes**: Setting up the basic parameters for the Electrolyser unit.
2. **Power Calculation Function**: Implementing a function to determine power input based on attributes like maximum capacity and demand profile.
3. **Developing advanced attributes**: Writing code to calculate the Electrolyser's efficiency based on its current power input.

**Understanding the Model**

The image below illustrates the concept of a simple Electrolyser unit model: 

![Capture.PNG](assume/docs/source//img/Electrolyser.PNG)


The image provides a visual representation of how dynamic efficiency varies based on different factors:

- **X-Axis**: Represents the varying power input to the Electrolyser unit.
- **Y-Axis**: Indicates the efficiency levels that correspond to different power inputs.
- **Curve**: Shows that the efficiency is not constant and varies depending on the current power input to the unit.

**Significance of this model**

Understanding this model is crucial for several reasons:

- **Adaptability**: The curve suggests that the unit can operate at different efficiency levels, allowing it to adapt to market conditions.
- **Optimization**: Knowing the efficiency levels at various power inputs allows the unit to operate at an optimal point, which is especially crucial in Demand Side Management (DSM) strategies.
- **Complexity**: The non-linear nature of the curve indicates that simple linear models may not be sufficient for capturing the unit's behavior, highlighting the need for a more complex model.

By understanding this model, you'll gain valuable insights into how to calculate and utilize dynamic efficiency in the Electrolyser unit, a crucial aspect of DSM in the ASSUME framework.

## 3.1 Initializing Core Attributes

The first step in our coding journey involves initializing the Electrolyser unit's core attributes. These include its maximum and minimum power capacities, hydrogen production levels, and other operational parameters. Let's take a look at the code snippet below:

In [3]:
# install jdc for some in line magic,
# that allows us defining functions of classes across different cells

!pip install jdc

In [4]:
# Import necessary modules
from datetime import datetime
import logging
import os
import pandas as pd
from assume import World
import jdc
from assume.common.base import SupportsMinMax, BaseStrategy
from assume.common.market_objects import MarketConfig, Order, Orderbook, Product

# Your Python code for initializing core attributes here

### 3.1.1 Initializing Core Attributes for a DSM Electrolyser Unit

Core attributes are essential parameters that define the operational behavior of the Electrolyser unit. These include:

- **ID**: A unique identifier for the unit.
- **Technology**: The type of technology used, which in this case is electrolysis for hydrogen production.
- **Unit Operator**: The entity responsible for operating the unit.
- **Bidding Strategies**: The strategies used by the unit for bidding in the electricity market.
- **Max Power and Min Power**: The maximum and minimum electrical power that the unit can handle.
- **Max Hydrogen and Min Hydrogen**: The maximum and minimum hydrogen production levels.
- **Fixed Cost**: The fixed operational cost for the unit.

**Why Are These Attributes Important?**

Understanding these attributes is crucial for the following reasons:

- They define the **range of actions** the Electrolyser unit can perform in the electricity market.
- They are used to **calculate dynamic efficiency** and **power input**, as we'll see in the later sections.
- They can be crucial for implementing **demand-side management strategies**, where the unit adjusts its operations based on market signals.

**Code Snippet for Initializing Core Attributes**

Here's how you would go about initializing these attributes:

In [5]:
# Initialize the Electrolyser class with core attributes

class Electrolyser(SupportsMinMax):
    def __init__(
        self,
        id: str,
        technology: str,
        index: pd.DatetimeIndex,
        unit_operator: str,
        bidding_strategies: str,
        max_power: float,
        min_power: float,
        max_hydrogen: float,
        min_hydrogen: float,
        fixed_cost: float,
        **kwargs,
    ):
        super().__init__(
            id=id,
            unit_operator=unit_operator,
            technology=technology,
            bidding_strategies=bidding_strategies,
            index=index,
            **kwargs,
        )

        self.min_hydrogen = min_hydrogen
        self.max_hydrogen = max_hydrogen

        self.max_power = max_power
        self.min_power = min_power
        self.ramp_up = ramp_up or max_power
        self.ramp_down = ramp_down or max_power
        self.fixed_cost = fixed_cost

        self.conversion_factors = self.get_conversion_factors()

In [6]:

# magic to enable class definitions across colab cells
%%add_to Electrolyser

def execute_current_dispatch(
        self,
        start: pd.Timestamp,
        end: pd.Timestamp,
    ):
        end_excl = end - self.index.freq

        # Calculate mean power for this time period
        avg_power = abs(self.outputs["energy"].loc[start:end_excl]).mean()

        # Decide which efficiency point to use
        if avg_power < self.min_power:
            self.outputs["energy"].loc[start:end_excl] = 0
            self.outputs["hydrogen"].loc[start:end_excl] = 0
        else:
            if avg_power <= 0.35 * self.max_power:
                dynamic_conversion_factor = self.conversion_factors[0]
            else:
                dynamic_conversion_factor = self.conversion_factors[1]

            self.outputs["energy"].loc[start:end_excl] = avg_power
            self.outputs["hydrogen"].loc[start:end_excl] = (
                avg_power / dynamic_conversion_factor
            )

        return self.outputs["energy"].loc[start:end_excl]

UsageError: Line magic function `%%add_to` not found.


## 3.2 Power Calculation Function

In this segment of the tutorial, we'll delve into the implementation of the power input function for the Electrolyser unit. This function calculates the amount of electrical power that should be supplied to the unit, taking into consideration several attributes like maximum capacity, demand profile, and dynamic efficiency.

The power input function is an algorithm that determines how much electrical power should be supplied to the Electrolyser unit at any given time. This function is integral for:

- **Optimizing Resource Utilization**: Ensuring that the Electrolyser operates within its optimal efficiency range.
- **Demand-Side Management (DSM)**: Allowing the unit to adapt its power consumption in response to market signals and constraints, thereby contributing to grid stability.

**Key Parameters Involved**

Here are some of the key parameters that the power input function will consider:

- **Maximum and Minimum Capacity**: The upper and lower bounds for power input to the Electrolyser.
- **Demand Profile**: The expected hydrogen production rates, which influence the power requirements.
- **Dynamic Efficiency**: The efficiency of converting power to hydrogen at different levels of power input, as discussed in the previous section.

**Code Snippet for Implementing the Power Input Function**

Now, let's take a look at the code snippet for implementing the power input function:

In [7]:
# magic to enable class definitions across colab cells
%%add_to Electrolyser

def calculate_min_max_power(
        self,
        start: pd.Timestamp,
        end: pd.Timestamp,
        hydrogen_demand=0,
    ):
        # check if hydrogen_demand is within min and max hydrogen production
        # and adjust accordingly
        if hydrogen_demand < self.min_hydrogen:
            hydrogen_production = self.min_hydrogen

        elif hydrogen_demand > self.max_hydrogen:
            hydrogen_production = self.max_hydrogen

        else:
            hydrogen_production = hydrogen_demand

        # get dynamic conversion factor
        dynamic_conversion_factor = self.get_dynamic_conversion_factor(
            hydrogen_production
        )
        power = hydrogen_production * dynamic_conversion_factor

        return power, hydrogen_production

UsageError: Line magic function `%%add_to` not found.


## 3.3 Developing Advanced Attributes

In this section of the tutorial, we'll guide you through the process of calculating the dynamic efficiency of the Electrolyser unit. Dynamic efficiency is a key performance metric that varies based on the unit's current power input.

### 3.3.1 What is Dynamic Efficiency?

Dynamic efficiency refers to the Electrolyser unit's ability to convert electrical power into hydrogen gas at varying rates of efficiency, depending on its current power input. This attribute is crucial for the following reasons:

- It allows the unit to **adapt to fluctuating market conditions**, optimizing its operation for price signals.
- It provides a **quantitative measure for decision-making**, particularly in DSM where the unit may need to adjust its demand profile.

### 3.3.2 Key Concepts for Dynamic Efficiency

Before we dive into the code, it's important to understand some key terms:

- **Average Power**: The mean power consumed during a specific time period.
- **Conversion Factors**: These are factors used to convert the average power into hydrogen production, and they can vary based on the power level.

**Code Snippet for Calculating Dynamic Efficiency**

Now let's take a look at how to implement dynamic efficiency in code. Below is a code snippet that you can use as a starting point:

In [8]:
# magic to enable class definitions across colab cells
%%add_to Electrolyser

def get_dynamic_conversion_factor(self, hydrogen_demand=None):
        # Adjust efficiency based on power
        if hydrogen_demand <= 0.35 * self.max_hydrogen:
            return self.conversion_factors[0]
        else:
            return self.conversion_factors[1]

def get_conversion_factors(self):
    # Calculate the conversion factor for the two efficiency points
    conversion_point_1 = (0.3 * self.max_power) / (
        0.35 * self.max_hydrogen
    )  # MWh / Tonne
    conversion_point_2 = self.max_power / self.max_hydrogen  # MWh / Tonne

    return [conversion_point_1, conversion_point_2]

UsageError: Line magic function `%%add_to` not found.


In [9]:
# magic to enable class definitions across colab cells
%%add_to Electrolyser

def as_dict(self) -> dict:
        unit_dict = super().as_dict()
        unit_dict.update(
            {
                "max_power": self.max_power,
                "min_power": self.min_power,
                "min_hydrogen": self.min_hydrogen,
                "max_hydrogen": self.max_hydrogen,
                "unit_type": "electrolyzer",
            }
        )
        return unit_dict

UsageError: Line magic function `%%add_to` not found.


# 4. Rule-Based Bidding Strategies

In this part of the tutorial, we formulate a rule-based bidding strategy for the Electrolyser unit. Rule-based strategies guide the unit's market participation by following a predefined set of rules and guidelines, enabling it to make market offers autonomously.

A rule-based bidding strategy is a set of algorithmic guidelines that dictate how a unit should participate in the electricity market. This is particularly important for:

- **Automated Decision-making**: Allows the unit to make bids autonomously, reducing the need for human intervention.
- **Market Adaptability**: Enables the unit to adjust its bidding behavior in response to dynamic market conditions.

**Key components of a Rule-Based Bidding Strategy**

To establish a rule-based bidding strategy, we need to consider several key components:

1. **Product Type**: This sets the stage for the kind of products (e.g., energy, ancillary services) that the unit will bid for in the electricity market.
2. **Bid Volume**: The quantity of the product that the unit offers in its bid.
3. **Marginal Revenue**: Used to determine the price at which the unit should make its bid. This involves calculating the unit's incremental revenue for additional units of electricity sold or consumed.

## 4.1 Defining NaiveStrategyElectrolyser Class

The `NaiveStrategyElectrolyser` class inherits from the `BaseStrategy` class and implements the `calculate_bids` method, which is responsible for formulating the market bids.

**Key Methods and Parameters**

- `calculate_bids`: This method takes several arguments, including the unit to be dispatched (`unit`), the market configuration (`market_config`), and a list of products (`product_tuples`). It returns an `Orderbook` containing the bids.

Here is the code:

In [10]:
class NaiveStrategyElectrolyser(BaseStrategy):
    """
    A naive strategy that bids the marginal cost of the electrolyser on the market.
    """

    def calculate_bids(
        self,
        unit: SupportsMinMax,
        market_config: MarketConfig,
        product_tuples: list[Product],
        **kwargs,
    ) -> Orderbook:
        """
        Takes information from a unit that the unit operator manages and
        defines how it is dispatched to the market

        :param unit: the unit to be dispatched
        :type unit: SupportsMinMax
        :param market_config: the market configuration
        :type market_config: MarketConfig
        :param product_tuples: list of all products the unit can offer
        :type product_tuples: list[Product]
        :return: the bids
        :rtype: Orderbook
        """
        start = product_tuples[0][0]  # start time of the first product
        bids = []

### 4.1.1 Calculating Marginal Revenue

Understanding the mathematical equation for calculating marginal revenue is crucial for grasping the economics of the bidding strategy. The formula used in the code is as follows:

$$\text{Marginal Revenue} = \left( \text{Hydrogen Price} - \text{Fixed Cost} \right) \times \frac{\text{Hydrogen Production}}{\text{Power}}$$


- **Hydrogen Price**: The price of hydrogen at the specific time frame, fetched from the unit's forecaster.
- **Fixed Cost**: The constant cost associated with the unit, not varying with the amount of power or hydrogen produced.
- **Hydrogen Production**: The amount of hydrogen produced during the given time frame, calculated based on the hydrogen demand.
- **Power**: The electrical power consumed by the Electrolyser unit to produce the given amount of hydrogen.

This mathematical equation provides just a mere example for understanding and implementing an effective rule-based bidding strategy for the Electrolyser unit within the ASSUME framework

**Calculating Bids for each Product**

In this section, we delve into the core of the bidding process, dissecting the loop that iterates over each product to calculate the bids. This loop serves as the crux of our rule-based bidding strategy.

The code starts by initializing an empty list, `bids`, and then proceeds to loop over each `product` in `product_tuples`.

Key Steps Inside the Loop

**1. Fetch Start and End Times:** For each product, the loop fetches the start and end times.

In [11]:
# magic to enable class definitions across colab cells
%%add_to NaiveStrategyElectrolyser

for product in product_tuples:
    """
    for each product, calculate the marginal revenue of the unit at the start time of the product
    and the volume of the product. Dispatch the order to the market.
    """

    start = product[0]
    end = product[1]

    # Calculate marginal cost using the adjusted efficiency
    hydrogen_demand = unit.forecaster[f"{unit.id}_h2demand"].loc[start]
    hydrogen_price = unit.forecaster[f"{unit.id}_h2price"].loc[start]

    power, hydrogen_production = unit.calculate_min_max_power(
        start=start,
        end=end,
        hydrogen_demand=hydrogen_demand,
    )

    marginal_revenue = (
        (hydrogen_price - unit.fixed_cost) * hydrogen_production / power
    )

    bid_price = marginal_revenue

    order: Order = {
        "start_time": product[0],
        "end_time": product[1],
        "only_hours": product[2],
        "price": bid_price,
        "volume": -power,
    }

    bids.append(order)

return bids

UsageError: Line magic function `%%add_to` not found.


**2. Retrieve Hydrogen Demand and Price:** The hydrogen demand and price are fetched from the unit's forecaster for the corresponding start time.

**3. Calculate Bid Volume:** The code calls unit.calculate_min_max_power to determine the bid volume based on the hydrogen demand.

**Calculate Marginal Revenue and Bid Price:** Using the formula discussed in the previous section, the marginal revenue is calculated to determine the bid price

### 4.1.2 Creating Market Orders and Finalizing Bids

In this segment of the tutorial, we shall explore how market orders are generated and added to the bids list. This step is crucial as it translates our calculated bid volume and price into actionable market orders.

**The `Order` Dictionary**

The `Order` dictionary object encapsulates all the necessary information for a single bid. It includes the following key-value pairs:

- **`start_time` and `end_time`**: The time window for which the bid is valid.
- **`volume`**: The bid volume, which is the power calculated based on hydrogen demand.
- **`price`**: The bid price, determined by the marginal revenue.

Here is how the `Order` dictionary is initialized:

    order: Order = {
                "start_time": product[0],
                "end_time": product[1],
                "only_hours": product[2],
                "price": bid_price,
                "volume": -power,
            }

            bids.append(order)

return bids

# 5. Integrating the New Unit and the New Rule Based Bidding Strategy in the World

After having developed the Electrolyser unit and the NaiveStrategyElectrolyser bidding strategy, the next vital step is to integrate these components into the ASSUME world.

**Initialize World Instance**

A new instance of the World class is created, which serves as the primary environment for running simulations.

In [12]:
# %% import packages
import logging
import os
from datetime import datetime, timedelta

import pandas as pd
from dateutil import rrule as rr

from assume import World
from assume.common.forecasts import CsvForecaster, NaiveForecast
from assume.common.market_objects import MarketConfig, MarketProduct

# %%
logger = logging.getLogger(__name__)

csv_path = "./outputs"
os.makedirs("./local_db", exist_ok=True)

# create world isntance
world = World(export_csv_path=csv_path)

# add new unit type to world
world.unit_types["electrolyser"] = Electrolyser
# add new bidding strategy to world
world.bidding_strategies["electrolyser_naive"] = NaiveStrategyElectrolyser


# %%
async def init():
    # define simulation period and ID
    start = datetime(2019, 1, 1)
    end = datetime(2019, 1, 31)
    index = pd.date_range(
        start=start,
        end=end + timedelta(hours=24),
        freq="H",
    )
    sim_id = "electrolyser_demo"

    # run world setup to create simulation and different roles
    # this creates the clock and the outputs role
    await world.setup(
        start=start,
        end=end,
        save_frequency_hours=None,
        simulation_id=sim_id,
        index=index,
    )

    # define market design and add it to a market
    marketdesign = [
        MarketConfig(
            name="EOM",
            opening_hours=rr.rrule(rr.HOURLY, interval=1, dtstart=start, until=end),
            opening_duration=timedelta(hours=1),
            market_mechanism="pay_as_clear",
            market_products=[
                MarketProduct(
                    duration=timedelta(hours=1),
                    count=1,
                    first_delivery=timedelta(hours=1),
                )
            ],
        )
    ]

    mo_id = "market_operator"
    world.add_market_operator(id=mo_id)
    for market_config in marketdesign:
        world.add_market(
            market_operator_id=mo_id,
            market_config=market_config,
        )

    # add unit operator
    world.add_unit_operator(id="power_plant_operator")

    # define a simple forecaster
    simple_forecaster = NaiveForecast(index, availability=1, fuel_price=0, co2_price=50)

    # add a unit to the world
    world.add_unit(
        id="power_plant_01",
        unit_type="power_plant",
        unit_operator_id="power_plant_operator",
        unit_params={
            "min_power": 0,
            "max_power": 100,
            "bidding_strategies": {"energy": "naive"},
            "fixed_cost": 5,
            "technology": "wind turbine",
        },
        forecaster=simple_forecaster,
    )

    # repeat for demand unit
    world.add_unit_operator("demand_operator")
    world.add_unit(
        id="demand_unit_1",
        unit_type="demand",
        unit_operator_id="demand_operator",
        unit_params={
            "min_power": 0,
            "max_power": 1000,
            "bidding_strategies": {"energy": "naive"},
            "technology": "demand",
        },
        forecaster=NaiveForecast(index, demand=50),
    )

    # load forecasts for hydrogen demand and hydrogen price
    hydrogen_forecasts = pd.read_csv(
        "assume/examples/inputs/example_03a/forecasts_df.csv",
        index_col=0,
        parse_dates=True,
    )

    # add the electrolyser unit to the world
    world.add_unit_operator(id="electrolyser_operator")
    hydrogen_plant_forecaster = CsvForecaster(index=index)
    hydrogen_plant_forecaster.set_forecast(data=hydrogen_forecasts)

    world.add_unit(
        id="elektrolyser_01",
        unit_type="electrolyser",
        unit_operator_id="electrolyser_operator",
        unit_params={
            "min_power": 7,
            "max_power": 52.25,
            "min_hydrogen": 0.15,
            "max_hydrogen": 0.95,
            "bidding_strategies": {"energy": "electrolyser_naive"},
            "technology": "electrolyser",
            "fixed_cost": 10,
        },
        forecaster=hydrogen_plant_forecaster,
    )


# %%
# run the simulation
world.loop.run_until_complete(init())
world.run()


AttributeError: 'NaiveForecast' object has no attribute 'name'